In [7]:
import collections
import queue
import random

In [2]:
Event = collections.namedtuple('Event', 'time proc action')

In [3]:
def taxi_process(ident, trips, start_time=0):
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home')

In [18]:
class Simulator:
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
    
    def run(self, end_time):
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)
        
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of events ***')
                break
            
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print('taxi: ', proc_id, proc_id * '  ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + random.randint(1, 30)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))

In [5]:
DEFAULT_INTERVAL = 5
num_taxis = 3
taxis = {i: taxi_process(i, (i + 1) * 2, i * DEFAULT_INTERVAL) for i in range(num_taxis)}

In [20]:
sim = Simulator(taxis)
sim.run(120)

taxi:  0  Event(time=0, proc=0, action='leave garage')
taxi:  1    Event(time=5, proc=1, action='leave garage')
taxi:  2      Event(time=10, proc=2, action='leave garage')
taxi:  0  Event(time=12, proc=0, action='pick up passenger')
taxi:  1    Event(time=20, proc=1, action='pick up passenger')
taxi:  1    Event(time=21, proc=1, action='drop off passenger')
taxi:  2      Event(time=25, proc=2, action='pick up passenger')
taxi:  0  Event(time=28, proc=0, action='drop off passenger')
taxi:  1    Event(time=33, proc=1, action='pick up passenger')
taxi:  2      Event(time=41, proc=2, action='drop off passenger')
taxi:  0  Event(time=50, proc=0, action='pick up passenger')
taxi:  2      Event(time=59, proc=2, action='pick up passenger')
taxi:  0  Event(time=63, proc=0, action='drop off passenger')
taxi:  1    Event(time=63, proc=1, action='drop off passenger')
taxi:  0  Event(time=66, proc=0, action='going home')
taxi:  1    Event(time=75, proc=1, action='pick up passenger')
taxi:  2      E